# Задача 6
## Случайная величина имеет распределение Парето: 

\begin{equation*} 
p(x) =
\begin{cases} 
\frac{θ-1}{x^θ}, & x  \geq 1 \\
0, & x < 1
\end{cases}
,θ > 1.
\end{equation*} 

### a) По выборке объёма n найти оценку параметра θ методом максимального правдоподобия.
### b) Построить доверительный интервал для медианы.
### c) Построить асимптотический доверительный интервал для параметра θ.

![](img/page_01.jpg)

![](img/page_02.jpg)

![](img/page_03.jpg)

![](img/page_04.jpg)

![](img/page_05.jpg)

### d) Сгенерируйте выборку объёма n = 100 для некоторого значения параметра θ. Вычислите указанные выше доверительные интервалы для доверительной вероятности 0.95.

In [2]:
import math
import pandas as pd
import random
import numpy as np
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

tetta = 10  # Недостижимая для меня оценка(
betta = 0.95


def p(x: float) -> float:
    return (tetta - 1) / x**tetta if x >= 1 else 0


def F(x: float) -> float:
    return 1 - 1 / x ** (tetta - 1) if x >= 1 else 0


def inverse_F(y: float) -> float:
    return 1 / ((1 - y) ** (1 / (tetta - 1)))


n = 100

sampling = pd.Series(inverse_F(random.uniform(0, 1)) for _ in range(n))  # Выборка

### Посчитаем асимптотический доверительный интервал

In [3]:
ln_mean = np.mean([np.log(x) for x in sampling])

t1 = -1.96
t2 = 1.96

lower_grade = 1 - (t2 - np.sqrt(n)) / (ln_mean * np.sqrt(n))
upper_grade = 1 - (t1 - np.sqrt(n)) / (ln_mean * np.sqrt(n))

console.print(f"Доверительный интервал: {lower_grade} < θ < {upper_grade}")
console.print(f"Длина интервала: {upper_grade - lower_grade}")

Доверительный интервал: 8.748345280562933 < θ < 12.526145467106058

Длина интервала: 3.777800186543125

### Посчитаем доверительный интервал для медианы

In [4]:
g_thetta_wave = 2 ** (ln_mean)

lower_grade = g_thetta_wave - t2 * g_thetta_wave * np.log(2) * ln_mean / np.sqrt(n)
upper_grade = g_thetta_wave - t1 * g_thetta_wave * np.log(2) * ln_mean / np.sqrt(n)

variation_row = sorted(sampling)

console.print(f"Медиана: {(variation_row[49] + variation_row[50])/2}")
console.print(f"Доверительный интервал: {lower_grade} < θ < {upper_grade}")
console.print(f"Длина интервала: {upper_grade - lower_grade}")

Медиана: 1.0874608764052232

Доверительный интервал: 1.0594251154739058 < θ < 1.0897217728168218

Длина интервала: 0.030296657342915978

### e) Численно постройте бутстраповский доверительный интервал двумя способами, используя параметрический бутстрап и непараметрический бутстрап.

### Посчитаем параметрический бутстраповский доверительный интервал

In [5]:
h_wave = 1 + 1 / ln_mean  # Оценка ОММП


def inverse_bootstrap_F(y: float, t: float) -> float:
    return 1 / ((1 - y) ** (1 / (t - 1)))


bootstrap_deltas = []

for _ in range(50000):
    bootstrap_sampling = pd.Series(
        inverse_bootstrap_F(random.uniform(0, 1), h_wave) for i in range(n)
    )
    bootstrap_ln_mean = np.mean([np.log(x) for x in bootstrap_sampling])
    bootstrap_h = 1 + 1 / bootstrap_ln_mean
    bootstrap_deltas.append(bootstrap_h - h_wave)

variation_row = sorted(bootstrap_deltas)

t1 = variation_row[int((1 - betta) * 50000 / 2)]
t2 = variation_row[int((1 + betta) * 50000 / 2)]

lower_grade = h_wave - t2
upper_grade = h_wave - t1


console.print(f"Доверительный интервал: {lower_grade} < θ < {upper_grade}")
console.print(f"Длина интервала: {upper_grade - lower_grade}")

Доверительный интервал: 8.431633615640175 < θ < 12.294018779946356

Длина интервала: 3.862385164306181

### Посчитаем непараметрический бутстраповский доверительный интервал

In [6]:
bootstrap_deltas = []

for _ in range(1000):
    subsampling = np.random.choice(sampling, size=n)
    ln_mean_bootstrap = np.mean([np.log(x) for x in subsampling])
    bootstrap_deltas.append(1 + 1 / ln_mean_bootstrap - h_wave)

variation_row = sorted(bootstrap_deltas)

t1 = variation_row[int((1 - betta) * 1000 / 2)]
t2 = variation_row[int((1 + betta) * 1000 / 2)]

lower_grade = h_wave - t2
upper_grade = h_wave - t1


console.print(f"Доверительный интервал: {lower_grade} < θ < {upper_grade}")
console.print(f"Длина интервала: {upper_grade- lower_grade}")

Доверительный интервал: 8.669748593717951 < θ < 12.226780925053363

Длина интервала: 3.557032331335412

### f) Сравнить все интервалы.
#### Длина непараметрического бутстраповского доверительного интервала < длина асимптотического доверительного интервала < длина параметрического бутстраповского доверительного интервала